In [48]:
import pandas as pd
import requests
from datetime import datetime

In [49]:
url = "https://datos.madrid.es/egob/catalogo/300107-0-agenda-actividades-eventos.json"
response = requests.get(url)
data = response.json()
evento_list = data['@graph']

In [50]:

eventos_list = {
    "nombre_evento": [],
    "url_evento": [],
    "codigo_postal": [],
    "direccion": [],
    "horario": [],
    "fecha_inicio": [],
    "fecha_fin": [],
    "organizacion": []
}

fecha_entrada = datetime.strptime("2025-03-01", "%Y-%m-%d").date()
fecha_salida = datetime.strptime("2025-03-01", "%Y-%m-%d").date()

for evento in evento_list:
    try:
        fecha_inicio = datetime.strptime(evento["dtstart"], "%Y-%m-%d %H:%M:%S.%f").date()
        fecha_fin = datetime.strptime(evento["dtend"], "%Y-%m-%d %H:%M:%S.%f").date()
    except KeyError:
        continue  
    except ValueError:  
        try:
            fecha_inicio = datetime.strptime(evento["dtstart"], "%Y-%m-%d %H:%M:%S").date()
            fecha_fin = datetime.strptime(evento["dtend"], "%Y-%m-%d %H:%M:%S").date()
        except Exception as e:
            continue  

    if fecha_inicio <= fecha_entrada and fecha_fin >= fecha_salida:
        nombre_evento = evento.get("title")
        url_evento = evento.get("link")
        horario = evento.get("time")

        if "address" in evento and "area" in evento["address"]:
            codigo_postal = evento["address"]["area"].get("postal-code")
            direccion = evento["address"]["area"].get("street-address")
        else:
            codigo_postal = "None"
            direccion = "None"

        organizacion = evento.get("organization", {}).get("organization-name")

        if nombre_evento and url_evento:  # Solo agregar si el evento tiene un nombre y URL
            eventos_list["nombre_evento"].append(nombre_evento)
            eventos_list["url_evento"].append(url_evento)
            eventos_list["codigo_postal"].append(codigo_postal)
            eventos_list["direccion"].append(direccion)
            eventos_list["horario"].append(horario)
            eventos_list["fecha_inicio"].append(fecha_inicio)
            eventos_list["fecha_fin"].append(fecha_fin)
            eventos_list["organizacion"].append(organizacion)

df_eventos = pd.DataFrame(eventos_list)


df_eventos.head(5)

,nombre_evento,url_evento,codigo_postal,direccion,horario,fecha_inicio,fecha_fin,organizacion
0,25º aniversario de la revista La Fragua,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28005,CALLE SAN JUSTO 5,,2025-02-24,2025-03-02,Biblioteca Pública Municipal Iván de Vargas (C...
1,60 Premio Reina Sofía de Pintura y Escultura,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28009,PASEO COLOMBIA 1,,2025-02-27,2025-03-23,Centro Cultural Casa de Vacas (Retiro)
2,A toda vela,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PLAZA LEGAZPI 8,,2024-10-01,2025-06-01,Matadero Madrid
3,Acompañamiento digital a personas mayores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,None,,2024-10-14,2025-06-30,None
4,Acompañar en la pérdida,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029,AVENIDA MONFORTE DE LEMOS 38,17:00,2025-01-14,2025-06-17,Biblioteca Pública Municipal José Saramago (Fu...


In [51]:
df_eventos.to_csv("../data/eventos.csv", index=False)